# Processing Iceberg Tables with Snowpark

## 1. Create Nation_Orders_Iceberg Table

In [ ]:
USE ROLE iceberg_lab;
USE DATABASE iceberg_lab;
USE SCHEMA iceberg_lab;
CREATE OR REPLACE ICEBERG TABLE nation_orders_iceberg (
    regionkey INTEGER,
    nationkey INTEGER,
    nation STRING,
    custkey INTEGER,
    order_count INTEGER,
    total_price INTEGER
)
    CATALOG = 'SNOWFLAKE'
    EXTERNAL_VOLUME = 'iceberg_lab_vol'
    BASE_LOCATION = '';

## 2. Use Snowpark to write data into Nation_Orders_Iceberg table

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import functions as sf

from snowflake.snowpark.context import get_active_session
session = get_active_session()

df_orders = session.read.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS")

# Perform aggregation on the DataFrame
df_orders_agg = (
    df_orders
    .groupBy(df_orders.o_custkey)
    .agg(
        sf.count(df_orders.o_orderkey).alias("order_count"),
        sf.sum(df_orders.o_totalprice).alias("total_price")
    )
)

df_orders_agg = df_orders_agg.select("o_custkey", "order_count", "total_price")

df_customer = session.read.table("ICEBERG_LAB.ICEBERG_LAB.CUSTOMER_ICEBERG")    
df_nation = session.read.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION")

df_nation_customer = df_customer.join(df_nation, df_customer.col("c_nationkey") == df_nation.col("n_nationkey")).select("c_custkey", df_nation["n_nationkey"].as_("nationkey"), df_nation["n_name"].as_("nation"), df_nation["n_regionkey"].as_("regionkey"))
df_nation_customer_orders_agg = df_nation_customer.join(df_orders_agg, df_nation_customer.col("c_custkey") == df_orders_agg.col("o_custkey")).select("regionkey", "nationkey", "nation", df_nation_customer["c_custkey"].as_("custkey"), "order_count", "total_price")

df_nation_customer_orders_agg = df_nation_customer_orders_agg.select("regionkey", "nationkey", "nation", "custkey", "order_count", "total_price")

# Save result to iceberg table
df_nation_customer_orders_agg.write.mode("append").save_as_table("nation_orders_iceberg")
df_nation_customer_orders_agg

In [ ]:
SELECT COUNT(*) FROM ICEBERG_LAB.ICEBERG_LAB.NATION_ORDERS_ICEBERG;